In [1]:
import pandas as pd
import requests as rq
from lxml import html
import numpy as np
from functools import reduce

In [2]:
def check_state(state):
    if state == "(noch) nicht behandelt":
        return False
    else:
        return True

In [3]:
stats ='//*[@id="antraege"]/li[position()>1]/ul/li/h4/span[@data-status]/@data-status'
appls ='//*[@id="antraege"]/li[position()>1]/ul/li/h4/a/text()'
j18 = "https://juso-buko.de/veranstaltungen/juso-bundeskongress-30-november-2-dezember-2018/"
j17 = "https://juso-buko.de/veranstaltungen/juso-bundeskongress-24-26-november-2017/"
URL_LIST = [j17, j18]

In [4]:
def get_site(url):
    r = rq.get(url)
    year = url[-5:-1]
    url_string = bytes(r.text, 'iso-8859-1').decode('utf-8')
    tree = html.document_fromstring(url_string)
    return year, tree

In [5]:
def forge_dataframe(year, tree):
    zy = tree.xpath(stats)
    zx = tree.xpath(appls)
    za = list(map(lambda x: x.split(" ", 1)[0], zx))
    zb = list(map(lambda x: "".join(x.split(" ", 1)[1]), zx))
    a = pd.DataFrame(np.column_stack([za, zb, zy]), columns=['id', 'title', 'status'])
    a["year"] = year
    return a

In [6]:
def get_data(url_list):
    df_list = []
    for url in url_list:
        year, tree = get_site(url)
        df_list.append(forge_dataframe(year, tree))
    df = pd.concat(df_list)
    return df

In [7]:
def perc_sum(a, b, c):
    return (a + b) / c

In [8]:
df = get_data(URL_LIST)

In [9]:
grp = df.groupby(["year", "status"]).count()
ac = grp.reset_index()
ad = ac.pivot(index='year', columns='status', values='id')
af = ad.rename(columns={"(noch) nicht behandelt": "notyet", "angenommen": "yes", "geändert angenommen": "changed"})

In [10]:
af["total"] = af[["notyet", "yes", "changed"]].sum(axis=1)
ag = af.reset_index()

In [11]:
ag["done"] = np.vectorize(perc_sum)(ag['yes'], ag['changed'], ag["total"])
ag["notdone"] = ag['notyet']/ ag["total"]

In [12]:
ag

status  year  notyet  yes  changed  total      done   notdone
0       2017     161    9       14    184  0.125000  0.875000
1       2018     132    6       26    164  0.195122  0.804878

In [13]:
antrags = df.loc[:]

In [14]:
antrags["state"] = antrags["status"].apply(check_state)

In [15]:
a = antrags.groupby("status").count().reset_index()

In [16]:
b = list(a["id"])

In [17]:
tot = reduce(lambda x, y: x+y, b)

In [18]:
per = list(map(lambda x: x[0] / x[1], list(zip(b, [tot, tot, tot]))))

In [19]:
list(map(lambda x: x*100, per))

[84.19540229885058, 4.310344827586207, 11.494252873563218]

In [20]:
non, yes, changed = b[0], b[1], b[2]

In [22]:
r = rq.get(URL_LIST[0])
url_string = bytes(r.text, 'iso-8859-1').decode('utf-8')
tree = html.document_fromstring(url_string)